In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as np
import pandas as pd

# tf tutorial
# https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/regression.ipynb#scrollTo=Cuym4yvk76vU

In [2]:
mat = loadmat('em_fixed_text_9loc.mat')
labels = mat['label']
traces = mat['traces']

raw_dataset = pd.DataFrame(np.concatenate((np.transpose(labels), traces), axis=1))
dataset = raw_dataset.copy()

In [ ]:
# Creat dataset 60/20/20
train_dataset = dataset.sample(frac = 0.8) # random shuffle
test_dataset = dataset.drop(train_dataset.index)

# Seperate lables
train_labels = train_dataset.pop(0)
test_labels = test_dataset.pop(0)
# train_dataset.describe()
# test_dataset.describe()

In [ ]:
inputs = keras.Input(shape=(5000,), name='sample_input')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(9, activation='softmax', name='Final')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='location')

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# model.summary()
keras.utils.plot_model(model, show_shapes=True)

In [110]:
# Train model
history = model.fit(train_dataset, train_labels, batch_size=64,
                   epochs=10, validation_split=0.25, verbose=2)

# Save model
model.save('tf_model_location.h5')

# Check accuracy on test set
test_loss, test_acc = newmodel.evaluate(test_dataset, test_labels)

Train on 5400 samples, validate on 1800 samples
Epoch 1/10
5400/5400 - 1s - loss: 6.0216e-04 - accuracy: 1.0000 - val_loss: 0.0128 - val_accuracy: 0.9961
Epoch 2/10
5400/5400 - 1s - loss: 5.2075e-04 - accuracy: 1.0000 - val_loss: 0.0116 - val_accuracy: 0.9972
Epoch 3/10
5400/5400 - 1s - loss: 3.6656e-04 - accuracy: 1.0000 - val_loss: 0.0111 - val_accuracy: 0.9972
Epoch 4/10
5400/5400 - 1s - loss: 2.8619e-04 - accuracy: 1.0000 - val_loss: 0.0127 - val_accuracy: 0.9972
Epoch 5/10
5400/5400 - 1s - loss: 3.0280e-04 - accuracy: 1.0000 - val_loss: 0.0126 - val_accuracy: 0.9967
Epoch 6/10
5400/5400 - 1s - loss: 2.8697e-04 - accuracy: 1.0000 - val_loss: 0.0122 - val_accuracy: 0.9972
Epoch 7/10
5400/5400 - 1s - loss: 2.2282e-04 - accuracy: 1.0000 - val_loss: 0.0120 - val_accuracy: 0.9972
Epoch 8/10
5400/5400 - 1s - loss: 1.8873e-04 - accuracy: 1.0000 - val_loss: 0.0127 - val_accuracy: 0.9972
Epoch 9/10
5400/5400 - 1s - loss: 1.8227e-04 - accuracy: 1.0000 - val_loss: 0.0118 - val_accuracy: 0.997

### Plot validation loss vs training loss 

In [ ]:
# hist = pd.DataFrame(history.history)
# hist['epoch'] = history.epoch
# hist.tail()
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.plot(hist['epoch'], hist['loss'],
           label='Training loss')
  plt.plot(hist['epoch'], hist['val_loss'],
           label = 'Val loss')
#   plt.ylim([0,1.1])
  plt.legend()
  plt.show()
plot_history(history)

### Load trained model 

In [105]:
# Load saved model
newmodel = keras.models.load_model('tf_model_location.h5')

### Sanity check of predictions on test set 

In [122]:
predictions = model.predict(test_dataset)

k = 123
predictions[k]

array([9.9981314e-01, 1.6256139e-04, 4.1772310e-09, 2.1282660e-05,
       1.7989589e-07, 8.5545156e-12, 6.1463135e-07, 2.1742560e-06,
       3.0277907e-09], dtype=float32)

In [162]:
model.predict(np.expand_dims(test_dataset.iloc[0], axis=0))

array([[9.9996758e-01, 2.7221689e-05, 8.6133006e-10, 4.5103129e-06,
        5.3060507e-08, 2.1812567e-12, 1.3737370e-07, 5.6239315e-07,
        1.5839262e-09]], dtype=float32)

In [144]:
# Location of highest prediction
np.argmax(predictions[0])

0

In [145]:
# Correct label in test set
test_labels.iloc[0]

0.0